# Feature Engineering Project1

#### Group Name: Unplugged
<h4>Group Members : Sumanth Ch, Amit R, Prasanna U</h4>

<b>Business Case:</b></br>

We will be preparing our own datasets about the various daily essential (or may be fast moving consumer good FMCG) products sold on various ecommerce platforms like (but not limited to) bigbasket, grofers, reliance smart, dmart, amazon pantry etc. The main objective of this project is to give you real life experience while doing data acquistion, data integration, data cleaning and data transforamtion before attempting any of the analytical activity on the data.

the four tasks that you will be doing as a part of this exercise will be as follows:</br>

Here are 3 ecommerce platforms used for data acquisition:
1. Spar India - https://www.sparindia.com/en/Grocery/c/10002
2. Dunzo(Ratnadeep) - https://www.dunzo.com/hyderabad/ratnadeep-super-market-begumpet?
3. Dailyneeds247 - https://www.dailyneeds247.com/buy-staples-online-supermarket-shopping-website/

<b>(A)Data acquisition:</b>
We gathered data from above 3 sources on the "groceries" category.

Steps are used to process the data is available in the jupyter notebook.

Each student got one dataset for each platform.


<b>(B)data cleaning</b>
Identifing the null, speacial characters and duplicate values.

Remove the no information gain data points.

Convert the type of the columns to perform calcualtions.

Derive or extract new columns to perform the calculations at aggregated levels.



<b>(C)Data integration</b>
Match the dataset columns

Integrate 3 datasets into one.

<b>(D)exploratory data analysis and recommendation</b>
Visualize the data to get the insights on the data.

Find outliers in case of any

<b>Use cases:>/b>
Discounted items availability in different platforms.
    
Select any product and quantity to get the price from different platforms.

Price ranges



In [876]:
# Import required Libraries
import json
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_colwidth=100

## Scrapping SPAR India Super Market
Steps:
1. Spar India has 26 pages with groceries data
2. By Looping these pages will get all products under the category.
3. There is title included with product name and quantity.
4. The actual product name text is not full loaded string.
5. Captured Product name,quantity, actual price and discount price.
6. Loaded all records into a dataframe.


In [877]:
def logic(url,params):
    
    #print(url,params)
    response = requests.get(url, params=params)# Get the response from the url
    
    soup = BeautifulSoup(response.text,'html.parser')# parse data
    #print(len(soup))
    results = soup.find_all('div',{'class':'row details'})#Finding the required dataset
    dataset = []
    
    for data in results:
        #product_name = data.h5.text.strip()
        product_name = data.find('div',{'class':'desc-selector'}).get('title').rsplit(" ", 1)[0]
        product_name = re.sub(' 200 gms| 1 kg| 1kg| 150 gms| 250x3 gms| 500 gms| 400 gms| 100 g| 250 g|250g| 180 gms| 120x48 gms| 50 gms', '', product_name)
        quantity = data.span.text.strip()
        Discounted_price = data.find('span',{'class':'orignal-price'}).text.replace('\t','')
        Discounted_price = Discounted_price.replace('\n','').replace('\rRs','')
        Actual_Price = data.find('span',{'class':'prev-price'})
        
        if Actual_Price:
            Actual_Price = Actual_Price.text
            Actual_Price = Actual_Price.replace('\n','').replace('\rRs','')
        else:
            Actual_Price = Discounted_price
        
        dataset.append((product_name,quantity,Discounted_price,Actual_Price))
        
        

    return dataset


#### Calling the scrapping function with page parameters. It will return a dataframe as a dataset.

In [878]:
if __name__=='__main__':
    rest = []
    params = {
            'q':':relevance_20001',
            'page':1
        }
    while params['page'] <= 26:#26 pages of data
        
        op = logic('https://www.sparindia.com/en/Grocery/c/10002?',params=params)
        #print(op)
        rest += op
        params['page'] += 1
        
    
    dataset = pd.DataFrame(rest)
dataset

,0,1,2,3
0,SPAR Ginger Garlic Paste,MRP Rs48.00,Rs48.00,Rs48.00
1,Spar Iodised Salt,MRP Rs16.00,Rs16.00,Rs16.00
2,Spar Whole Wheat Chakki Fresh Atta,MRP Rs230.00,Rs230.00,Rs230.00
3,Spar - Loose Value Idli Rice,1kg,56.00,56.00
4,Bongo Bhog - Royal Minikit Rice,5kg,400.00,400.00
...,...,...,...,...
531,Spar Iodised Salt,MRP Rs16.00,Rs16.00,Rs16.00
532,Spar Whole Wheat Chakki Fresh Atta,MRP Rs230.00,Rs230.00,Rs230.00
533,SPAR Ginger Garlic Paste,MRP Rs48.00,Rs48.00,Rs48.00
534,Spar Iodised Salt,MRP Rs16.00,Rs16.00,Rs16.00


In [879]:
# Adding column names to raw dataset
dataset.columns = ['Product_Name','Quantity','Discounted_Price','Actual_Price']
# Getting dataset into a raw dataset without disturbign the actual one.
raw_data = dataset
# Sample to look at the data
raw_data.head(50)

,Product_Name,Quantity,Discounted_Price,Actual_Price
0,SPAR Ginger Garlic Paste,MRP Rs48.00,Rs48.00,Rs48.00
1,Spar Iodised Salt,MRP Rs16.00,Rs16.00,Rs16.00
2,Spar Whole Wheat Chakki Fresh Atta,MRP Rs230.00,Rs230.00,Rs230.00
3,Spar - Loose Value Idli Rice,1kg,56.00,56.00
4,Bongo Bhog - Royal Minikit Rice,5kg,400.00,400.00
5,R K- Kolkata Govind Bhog Rice,1kg,140.00,140.00
6,Bongo Bhog - Premium Aromatic Govind Bhog Rice,5kg,625.00,625.00
7,Organic Tattva - Idli Rice,1kg,130.00,130.00
8,Bongo - Bhog Premium Aromatic Govind Bhog Rice,1kg,150.00,150.00
9,Safe Harvest Sona Masuri Unpolished Brown Rice,1kg,90.00,90.00


## Cleansing SPAR India Dataset
<b>Steps:</b></br>

There are records from advertising which doesn't add any information to the data and will duplicated for actual data. Those records to be removed from dataset.

Remove the special characters in any columns data.

Convert the quantity and price columns to int or float to perform calculations.

Remove comma from proce columns as few products may have comma seperated prices.

Convert the quantity to the lowest granular like gram/ml/unit to have a normalized data.

Create a new feature discount %.

Extract a new feature to have the price of each product at a lowest possible granularity.


In [880]:
# Take out the advertisements data which doesn't have any information which doesn't add values to dataset
raw_data1 = raw_data[~raw_data['Quantity'].str.contains("MRP")]
raw_data1.reset_index(drop=True, inplace=True)
# Clean special characters in the dataset
raw_data1['Actual_Price'] = raw_data1['Actual_Price'].str.replace('\r\t\t\t\t\t\t\t\t\t\t\tMRP Rs','')
raw_data1

,Product_Name,Quantity,Discounted_Price,Actual_Price
0,Spar - Loose Value Idli Rice,1kg,56.00,56.00
1,Bongo Bhog - Royal Minikit Rice,5kg,400.00,400.00
2,R K- Kolkata Govind Bhog Rice,1kg,140.00,140.00
3,Bongo Bhog - Premium Aromatic Govind Bhog Rice,5kg,625.00,625.00
4,Organic Tattva - Idli Rice,1kg,130.00,130.00
...,...,...,...,...
453,Turn Organic - Chicken Seasoning,60g,199.00,199.00
454,24 Mantra - Organic Wheat Grass Powder,100g,290.00,290.00
455,Spar - Brewed Natural Apple Cider Vinegar,370ml,199.00,390.00
456,Pro Nature - Organic Apple Cider Vinegar,500ml,560.00,560.00


In [881]:
# raw data types
raw_data1.dtypes

Product_Name        object
Quantity            object
Discounted_Price    object
Actual_Price        object
dtype: object

In [882]:
#Convert Pricing column to float datatype as we ma need to perform some calcualtions in future.
# Replaced Comma seperator
# Converted the actual_price and dicounted_price columns to float.
raw_data1['Actual_Price'].unique()
raw_data1['Discounted_Price'].unique()
raw_data1['Actual_Price']= raw_data1['Actual_Price'].str.replace(",","").astype(float)
raw_data1['Discounted_Price']= raw_data1['Discounted_Price'].str.replace(",","").astype(float)      
raw_data1.dtypes

Product_Name         object
Quantity             object
Discounted_Price    float64
Actual_Price        float64
dtype: object

In [883]:
#Replaced/converted the Quantity values to possible values from data
#raw_data1['Quantity'] = raw_data1['Quantity'].replace({'1N':'0.5','120x48':'120','250x3':'750','1.2kg':'1200',' kg':'','kg':'000',' gms':'',' gm':'','g':'','ml':'','L':'000',' litre':'000','1U':'300',}, regex=True)
raw_data1['Quantity'] = raw_data1['Quantity'].replace({'1N':'0.5','120x48':'120','250x3':'750','1.2kg':'1200',' kg':'','kg':'000',' gms':'',' gm':'','g':'','ml':'','L':'000',' litre':'000','1U':'300',}, regex=True)
raw_data1 = raw_data1.sort_values(by='Quantity')

In [884]:
# Converted the Quantity to float and cleared the data issues
raw_data1['Quantity']= raw_data1['Quantity'].astype(float)
raw_data1.loc[raw_data1.Product_Name == "Safe Harvest - Rice Flour 1 kg,", 'Quantity'] = 1000
raw_data1.loc[raw_data1.Product_Name == "POPULAR URAD DAL", 'Quantity'] = 1000
raw_data1.loc[raw_data1.Product_Name == "POPULAR TUR DAL", 'Quantity'] = 2000

In [885]:
# Created or derived a new column discount from existing columns.
raw_data1['Discount %'] = (((raw_data1['Actual_Price'] - raw_data1['Discounted_Price'])/raw_data1['Actual_Price'])*100).round(2)
raw_data1.sort_values(by='Discount %',ascending=False)

,Product_Name,Quantity,Discounted_Price,Actual_Price,Discount %
43,Elina Long Grain Rice,1000.0,70.0,160.0,56.25
455,Spar - Brewed Natural Apple Cider Vinegar,370.0,199.0,390.0,48.97
401,Disano - Extra Light Olive Oil,2000.0,1399.0,2395.0,41.59
400,Disano - Extra Light Olive Oil,1000.0,749.0,1250.0,40.08
265,Keya - Dhaabe Da Keema Masala,100.0,84.0,120.0,30.00
...,...,...,...,...,...
139,"Indiras - Tamarind Concentrate,",150.0,58.0,58.0,0.00
126,Tata Nx - Zero Sugar,120.0,275.0,275.0,0.00
183,Aashirvaad Whole Wheat Atta,10000.0,490.0,490.0,0.00
2,R K- Kolkata Govind Bhog Rice,1000.0,140.0,140.0,0.00


In [886]:
# Created new column to have the price of the column at unit of ONE
raw_data1['Price per gram'] = (raw_data1['Discounted_Price'] / raw_data1['Quantity']).round(2)
raw_data1.tail(50)

,Product_Name,Quantity,Discounted_Price,Actual_Price,Discount %,Price per gram
349,Best Price Loose Cashew Whole,500.0,600.0,600.0,0.00,1.20
348,Spar - Almonds,500.0,590.0,590.0,0.00,1.18
347,Spar - Cashewnut,500.0,700.0,700.0,0.00,1.40
30,Safe Harvest Foxtail Millet Rice,500.0,77.0,77.0,0.00,0.15
31,Safe Harvest Kodo Millet Rice,500.0,77.0,77.0,0.00,0.15
27,"Spar - Select Amaranth,",500.0,87.0,87.0,0.00,0.17
3,Bongo Bhog - Premium Aromatic Govind Bhog Rice,5000.0,625.0,625.0,0.00,0.12
120,Mawana Sugar,5000.0,285.0,300.0,5.00,0.06
7,Best Price Loose Palakkad Matta Boiled Rice,5000.0,290.0,290.0,0.00,0.06
423,24 Mantra - Sona Masuri Rice Organic,5000.0,425.0,475.0,10.53,0.08


In [901]:
# Added store_name and ecommerce platform information to dataset
empty_columns = pd.DataFrame(columns=['Store_Name','URL'])
ds  = pd.concat([raw_data1,empty_columns],ignore_index= True)
ds['Store_Name'] = ds['Store_Name'].replace(np.nan,'Spar India',regex=True)
ds['URL'] = ds['URL'].replace(np.nan,'https://www.sparindia.com/',regex=True)
ds.columns = ['Product_Name','Prod_Quantity','Product_Discounted_Price','Product_Price','Discount_%','Price_Per_Unit','Store_Name','Links']
dataset1 = ds

In [902]:
dataset1.head()

,Product_Name,Prod_Quantity,Product_Discounted_Price,Product_Price,Discount_%,Price_Per_Unit,Store_Name,Links
0,Pina - Saffron,0.5,349.0,349.0,0.00,698.00,Spar India,https://www.sparindia.com/
1,"Safe Harvest - Rice Flour,",1.0,69.0,69.0,0.00,69.00,Spar India,https://www.sparindia.com/
2,POPULAR URAD DAL,1000.0,148.0,150.0,1.33,0.15,Spar India,https://www.sparindia.com/
3,Taj Mahal - Spanish Saffron,1.0,605.0,605.0,0.00,605.00,Spar India,https://www.sparindia.com/
4,Kesari - Saffron Threads,1.0,375.0,375.0,0.00,375.00,Spar India,https://www.sparindia.com/


## Loaded dataset into a CSV File
File will store under the same location of .pynb file.

In [891]:
dataset1.to_csv('SparIndia_groceries.csv', encoding='utf-8')

## Scrapping Rantadeep Super Market from Dunzo
Steps:
1. Rantadeep Super Market with groceries data
2. All products can be scrapped from one url
3. Different HTML classes to capture the each attribute of the product.
4. Captured Product name,quantity, price 
5. Loaded all records into a dataframe.

In [933]:
# Opening the url
url = "https://www.dunzo.com/hyderabad/ratnadeep-super-market-begumpet?subTag=fnv"
html = urlopen(url)
print(html)

In [934]:
#parse the data
soup = BeautifulSoup(html, 'lxml')
print(soup)

<!DOCTYPE html>
<html lang="en"><head><meta content="website" name="og:type"/><meta content="https://ik.imagekit.io/dunzo/web-assets/images/dunzo-logo-2ff5b2c3b8feac1dffa4b54a09822186.png" property="og:image"/><meta content="app" name="twitter:card"/><meta content="@Dunzoit" name="twitter:site"/><meta content="@Dunzoit" name="twitter:creator"/><meta content="in" name="twitter:app:country"/><meta content="https://ik.imagekit.io/dunzo/web-assets/images/dunzo-logo-2ff5b2c3b8feac1dffa4b54a09822186.png" name="twitter:image"/><meta content="#ffffff" name="theme-color"/><meta content="yes" name="mobile-web-app-capable"/><meta content="Dunzo" name="apple-mobile-web-app-title"/><meta content="yes" name="apple-mobile-web-app-capable"/><meta content="default" name="apple-mobile-web-app-status-bar-style"/><meta content="Dunzo" name="application-name"/><meta content="Dunzo" name="msapplication-tooltip"/><meta content="/" name="msapplication-starturl"/><meta content="no" name="msapplication-tap-high

In [935]:
title = soup.title
print(title)

<title class="next-head">Order from Ratnadeep Super Market online in Hyderabad | Dunzo</title>


In [936]:
#get the text
text = soup.get_text()
text

'LUX=(function(){var a=("undefined"!==typeof(LUX)&&"undefined"!==typeof(LUX.gaMarks)?LUX.gaMarks:[]);var d=("undefined"!==typeof(LUX)&&"undefined"!==typeof(LUX.gaMeasures)?LUX.gaMeasures:[]);var j="LUX_start";var k=window.performance;var l=("undefined"!==typeof(LUX)&&LUX.ns?LUX.ns:(Date.now?Date.now():+(new Date())));if(k&&k.timing&&k.timing.navigationStart){l=k.timing.navigationStart}function f(){if(k&&k.now){return k.now()}var o=Date.now?Date.now():+(new Date());return o-l}function b(n){if(k){if(k.mark){return k.mark(n)}else{if(k.webkitMark){return k.webkitMark(n)}}}a.push({name:n,entryType:"mark",startTime:f(),duration:0});return}function m(p,t,n){if("undefined"===typeof(t)&&h(j)){t=j}if(k){if(k.measure){if(t){if(n){return k.measure(p,t,n)}else{return k.measure(p,t)}}else{return k.measure(p)}}else{if(k.webkitMeasure){return k.webkitMeasure(p,t,n)}}}var r=0,o=f();if(t){var s=h(t);if(s){r=s.startTime}else{if(k&&k.timing&&k.timing[t]){r=k.timing[t]-k.timing.navigationStart}else{return}

In [937]:
# Read the data based on the class
table = soup.findAll('div',attrs={"class":"kYmYWE"})
d = []
dataset = pd.DataFrame(columns=["Product_Name", "Product_Price", "Product_Qty"])

for div in table:
    product_name = div.find('p', attrs={"class":"eCmpNP"}).text
    product_price = div.find('p', attrs={"class":"iNqLHC"}).text
    
    if(div.find('p', attrs={"class":"gaxUzt"})) is not None:
        product_qty = div.find('p', attrs={"class":"gaxUzt"}).text
        #print(product_qty)
    else:
        product_qty = div.find('span', attrs={"class":"gaxUzt"}).text
    row = [product_name, product_price, product_qty];
    d.append(row)
    
dataset = pd.DataFrame(d, columns=["Product_Name", "Product_Price", "Product_Qty"])
#load the data into dataset 
print(dataset)

                      Product_Name Product_Price Product_Qty
0                        Baby Corn          ₹ 59    1 Packet
1                         Broccoli         ₹ 139        1 Kg
2                     Red Capsicum         ₹ 130        1 Kg
3                Fresh Broad Beans          ₹ 53     500 Gms
4                       Raw Papaya          ₹ 35        1 Kg
..                             ...           ...         ...
658          Deepam Lamp Pooja Oil         ₹ 220      900 Ml
659                        Vathulu          ₹ 15       5 Pcs
660  Sri Somasila Jb Gatti Vattulu          ₹ 17     100 Pcs
661               Ratnadeep Kumkum          ₹ 10      50 Gms
662                Ratnadeep Loban          ₹ 40      50 Gms

[663 rows x 3 columns]


## Ratnadeep super market data cleansing


In [938]:
# Added a new column to store name to differentiate records from other datasets
dataset['Store_Name'] = "Ratnadeep"

In [939]:
#length of records
print(len(dataset))
dataset.head(100)

663


,Product_Name,Product_Price,Product_Qty,Store_Name
0,Baby Corn,₹ 59,1 Packet,Ratnadeep
1,Broccoli,₹ 139,1 Kg,Ratnadeep
2,Red Capsicum,₹ 130,1 Kg,Ratnadeep
3,Fresh Broad Beans,₹ 53,500 Gms,Ratnadeep
4,Raw Papaya,₹ 35,1 Kg,Ratnadeep
...,...,...,...,...
95,Ratnadeep Double Beans,₹ 30,200 Gms,Ratnadeep
96,Ratnadeep Kashmir Rajma,₹ 64,500 Gms,Ratnadeep
97,Ratnadeep Chitra Rajma,₹ 137,1 Kg,Ratnadeep
98,Ratnadeep Idly Rava,₹ 83,2 Kgs,Ratnadeep


In [940]:
print(dataset[dataset['Product_Name'] == ''])
print(dataset[dataset['Product_Price'] == ''])
print(dataset[dataset['Product_Qty'] == ''])

Empty DataFrame
Columns: [Product_Name, Product_Price, Product_Qty, Store_Name]
Index: []
Empty DataFrame
Columns: [Product_Name, Product_Price, Product_Qty, Store_Name]
Index: []
Empty DataFrame
Columns: [Product_Name, Product_Price, Product_Qty, Store_Name]
Index: []


In [941]:
# Identify in case of any null records
print(pd.isnull(dataset['Product_Name']).sum())
print(pd.isnull(dataset['Product_Price']).sum())
print(pd.isnull(dataset['Product_Qty']).sum())

0
0
0


#### A regular expression function to convert the quantity values to handle in one shot

In [942]:
r = re.compile("^[0-9]+\.?[0-9]+$ ([a-zA-Z]+)")
prod_qty_data = dataset['Product_Qty']
a = []
b = []

prod_qty = {"Prod_Quantity":[], "Prod_Units":[]} 
for item in dataset.index:
    qty_str = dataset['Product_Qty'][item]
    res = qty_str.split();
    if(len(res) <= 2):
        prod_quantity = res[0]
        if(len(res) > 1):
            prod_unit = res[1]
        else:
            prod_unit = ''
    else:
        prod_quantity = res[0]
        prod_unit = res[1]
    a.append(prod_quantity)
    b.append(prod_unit)
dataset['Prod_Quantity'] = a
dataset['Prod_Units'] = b

In [943]:
dataset.head(10)

,Product_Name,Product_Price,Product_Qty,Store_Name,Prod_Quantity,Prod_Units
0,Baby Corn,₹ 59,1 Packet,Ratnadeep,1,Packet
1,Broccoli,₹ 139,1 Kg,Ratnadeep,1,Kg
2,Red Capsicum,₹ 130,1 Kg,Ratnadeep,1,Kg
3,Fresh Broad Beans,₹ 53,500 Gms,Ratnadeep,500,Gms
4,Raw Papaya,₹ 35,1 Kg,Ratnadeep,1,Kg
5,Britannia Brown Bread,₹ 40,450 Gms,Ratnadeep,450,Gms
6,Britannia Healthy Slice Bread,₹ 42,450 Gms,Ratnadeep,450,Gms
7,Britannia Vitarich Bread,₹ 35,400 Gms,Ratnadeep,400,Gms
8,Fresh Bakes Burger Bun,₹ 35,200 Gms,Ratnadeep,200,Gms
9,Fresh Bakes Fruit Bread,₹ 20,200 Gms,Ratnadeep,200,Gms


In [944]:
len(dataset)

663

In [945]:
# Currency symbol removed from the price column
dataset['Product_Price'] = dataset['Product_Price'].str[2:]

In [946]:
dataset.head(10)

,Product_Name,Product_Price,Product_Qty,Store_Name,Prod_Quantity,Prod_Units
0,Baby Corn,59,1 Packet,Ratnadeep,1,Packet
1,Broccoli,139,1 Kg,Ratnadeep,1,Kg
2,Red Capsicum,130,1 Kg,Ratnadeep,1,Kg
3,Fresh Broad Beans,53,500 Gms,Ratnadeep,500,Gms
4,Raw Papaya,35,1 Kg,Ratnadeep,1,Kg
5,Britannia Brown Bread,40,450 Gms,Ratnadeep,450,Gms
6,Britannia Healthy Slice Bread,42,450 Gms,Ratnadeep,450,Gms
7,Britannia Vitarich Bread,35,400 Gms,Ratnadeep,400,Gms
8,Fresh Bakes Burger Bun,35,200 Gms,Ratnadeep,200,Gms
9,Fresh Bakes Fruit Bread,20,200 Gms,Ratnadeep,200,Gms


In [947]:
# Identify different product quantity units available in the dataset.
dataset['Prod_Units'].unique()

array(['Packet', 'Kg', 'Gms', 'Eggs', 'Pcs', 'Ml', 'Ltr', 'x', 'Kgs',
       'kg', 'Ltrs', '', 'Pc', 'Gm', 'of', 'Pads', 'Sheets', 'Pulls',
       'Sticks'], dtype=object)

In [948]:
# Identify different product quantities available in the dataset.
dataset['Prod_Quantity'].unique()

array(['1', '500', '450', '400', '200', '150', '30', '6', '250', '100',
       '480', '85', '90', '180', '300', '26', '1.5', '60', '325', '5',
       '2', '800', '900', '50', '20', '25', '15', '170', '290', '700',
       '65', '70', '420', '73', '350', '225', '390', '40', '12', '175',
       '17', '16', '11', '600Ml', '750', '1.75', '240', '1.2', '210',
       '330', '64', '120', '75', '46.5', '53', '52', '110', '80', '160',
       '45', '55', '95', '59', '28', '23', '130', '47', '145', '33', '58',
       '8.9', '17.8', '56', '13.2', '18', '36', '227', '68.25', '51', '3',
       '43', '18.69', '13', '27.5', '38.3', '28.7', '72', '10', '33.6',
       '66', '115', '125', 'Pack', '7', 'Large'], dtype=object)

In [949]:
# Identifying the count w.r.t different quantity types to convert them to a single unit
# Most of the products gms,Kg,MI etc.
dataset.groupby('Prod_Units').count()

,Product_Name,Product_Price,Product_Qty,Store_Name,Prod_Quantity
Prod_Units,,,,,
,2,2,2,2,2
Eggs,1,1,1,1,1
Gm,1,1,1,1,1
Gms,458,458,458,458,458
Kg,49,49,49,49,49
Kgs,11,11,11,11,11
Ltr,24,24,24,24,24
Ltrs,7,7,7,7,7
Ml,77,77,77,77,77


In [950]:
# Check how many products are from the store brand .
dataset[dataset['Product_Name'].str.contains("Ratnadeep")]

,Product_Name,Product_Price,Product_Qty,Store_Name,Prod_Quantity,Prod_Units
12,Ratnadeep Bread Crumbs,24,200 Gms,Ratnadeep,200,Gms
60,Ratnadeep Soya Granules,54,300 Gms,Ratnadeep,300,Gms
70,Ratnadeep Barley,12,200 Gms,Ratnadeep,200,Gms
71,Ratnadeep Foxtail Millets,150,1 Kg,Ratnadeep,1,Kg
72,Ratnadeep Idli Rice,240,5 Kgs,Ratnadeep,5,Kgs
...,...,...,...,...,...,...
654,Ratnadeep Silver Coated Plates No.12,102,25 Pcs,Ratnadeep,25,Pcs
655,Ratnadeep Silver Coated Plates No.10,78,25 Pcs,Ratnadeep,25,Pcs
657,Ratnadeep Pacha Karpooram,80,25 Gms,Ratnadeep,25,Gms
661,Ratnadeep Kumkum,10,50 Gms,Ratnadeep,50,Gms


In [951]:
# Different varieties of Basmati Rice
dataset[dataset['Product_Name'].str.contains("Basmati")]

,Product_Name,Product_Price,Product_Qty,Store_Name,Prod_Quantity,Prod_Units
68,Daawat Super Basmati Rice,165,1 Kg,Ratnadeep,1,Kg
81,Ratnadeep Basmati Rice,89,1 Kg,Ratnadeep,1,Kg


In [952]:
# Differnet products available for Pista
dataset[dataset['Product_Name'].str.contains("Pista")]

,Product_Name,Product_Price,Product_Qty,Store_Name,Prod_Quantity,Prod_Units
269,Ratnadeep Green Pista,235,100 Gms,Ratnadeep,100,Gms
270,Ratnadeep Pista Salted,675,500 Gms,Ratnadeep,500,Gms
409,Britannia Good Day Pista Badam Cookies,20,100 Gms,Ratnadeep,100,Gms
424,Unibic Pista Badam Cookies,25,75 Gms,Ratnadeep,75,Gms
427,Britannia Good Day Pista Cashew Almond Cookies,40,200 Gms,Ratnadeep,200,Gms


In [953]:
# Identifing data types
dataset.dtypes

Product_Name     object
Product_Price    object
Product_Qty      object
Store_Name       object
Prod_Quantity    object
Prod_Units       object
dtype: object

In [954]:
# Identifying odd records
print(dataset[dataset['Prod_Quantity'] == '600Ml'])
print(dataset[dataset['Prod_Quantity'] == 'Pack'])
print(dataset[dataset['Prod_Quantity'] == 'Large'])

                Product_Name Product_Price Product_Qty Store_Name  \
349  Mountain Dew Pet Bottle            35       600Ml  Ratnadeep   

    Prod_Quantity Prod_Units  
349         600Ml             
                   Product_Name Product_Price          Product_Qty Store_Name  \
605  Pears Pure And Gentle Soap           259  Pack of 5 x 125 Gms  Ratnadeep   

    Prod_Quantity Prod_Units  
605          Pack         of  
               Product_Name Product_Price Product_Qty Store_Name  \
644  Plastobag Garbage Bags            99       Large  Ratnadeep   

    Prod_Quantity Prod_Units  
644         Large             


In [955]:
# Dropping odd records with mismatch in teh quantity. Dropping these records will not impact the data instead 
#these records may lead to different or false recommendations.
dataset.drop(dataset.index[[349, 605, 644]], inplace=True)

In [956]:
# Converting the price and quantity to numeric
dataset[['Product_Price', 'Prod_Quantity']] = dataset[['Product_Price', 'Prod_Quantity']].apply(pd.to_numeric) 

In [957]:
dataset.head(5)

,Product_Name,Product_Price,Product_Qty,Store_Name,Prod_Quantity,Prod_Units
0,Baby Corn,59,1 Packet,Ratnadeep,1.0,Packet
1,Broccoli,139,1 Kg,Ratnadeep,1.0,Kg
2,Red Capsicum,130,1 Kg,Ratnadeep,1.0,Kg
3,Fresh Broad Beans,53,500 Gms,Ratnadeep,500.0,Gms
4,Raw Papaya,35,1 Kg,Ratnadeep,1.0,Kg


In [958]:
# Converting differnet quantity types to one
grams = ["Gms", "Gm", "Ml", "Pc", "Pcs"]
kilos = ["Kgs", "Kg", "kg", "Ltrs", "Ltr"]
dataset['Price_Per_Unit'] = ''

for item in dataset.index:
    prod_qty = dataset['Prod_Quantity'][item]
    prod_unit = dataset['Prod_Units'][item]
    prod_price = dataset['Product_Price'][item]
    if any(x in prod_unit for x in kilos):
        dataset['Price_Per_Unit'][item] = prod_price/(prod_qty * 1000)
    elif(x in prod_unit for x in grams):
        dataset['Price_Per_Unit'][item] = prod_price/(prod_qty)
    else:
        dataset['Price_Per_Unit'][item] = prod_price

In [964]:
# Rounding entire dataset values to 3
dataset[['Price_Per_Unit']] = dataset[['Price_Per_Unit']].apply(pd.to_numeric)
dataset2 = dataset

In [965]:
# Loading dataset into csv file.
dataset2.to_csv('ratnadeep.csv', index=False, encoding='utf-8')

In [966]:
dataset2.head(20)

,Product_Name,Product_Price,Product_Qty,Store_Name,Prod_Quantity,Prod_Units,Price_Per_Unit
0,Baby Corn,59,1 Packet,Ratnadeep,1.0,Packet,59.000000
1,Broccoli,139,1 Kg,Ratnadeep,1.0,Kg,0.139000
2,Red Capsicum,130,1 Kg,Ratnadeep,1.0,Kg,0.130000
3,Fresh Broad Beans,53,500 Gms,Ratnadeep,500.0,Gms,0.106000
4,Raw Papaya,35,1 Kg,Ratnadeep,1.0,Kg,0.035000
5,Britannia Brown Bread,40,450 Gms,Ratnadeep,450.0,Gms,0.088889
6,Britannia Healthy Slice Bread,42,450 Gms,Ratnadeep,450.0,Gms,0.093333
7,Britannia Vitarich Bread,35,400 Gms,Ratnadeep,400.0,Gms,0.087500
8,Fresh Bakes Burger Bun,35,200 Gms,Ratnadeep,200.0,Gms,0.175000
9,Fresh Bakes Fruit Bread,20,200 Gms,Ratnadeep,200.0,Gms,0.100000


# Dailyneeds 247 web scrapping

In [977]:
#dailyneeds Scrapping
def search_dailyneeds247():
    dailyneeds_url = 'https://www.dailyneeds247.com/buy-staples-online-supermarket-shopping-website/'
    #Necessary request headers
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    html = urlopen(dailyneeds_url)
    #load url using Beautifulsoup
    soup = BeautifulSoup(html, 'lxml')
    table = soup.findAll('div',attrs={"class":"uael-woo-products-summary-wrap"})
    d = []
    dataset = pd.DataFrame(columns=["Product_Name", "Product_Price", "Product_Qty"])
    #Extract product data into a dictionary object
    for div in table:
        product_name = div.find('h2', attrs={"class":"woocommerce-loop-product__title"}).text
        product_price = div.find('span', attrs={"class":"woocommerce-Price-amount amount"}).text
        qty_split = product_name.split(' ')
        prod_quantity = qty_split[-2]
        prod_units = qty_split[-1]
        product_qty = prod_quantity+' '+prod_units
        stor_name="Dailyneeds247"
        price_per_unit=''
        product_price = product_price.replace('₹','').replace(',','')
        #if any(x in product_units for x in kilos):
        #    price_per_unit = int(product_price.replace('₹','').replace(',',''))/int(product_quantity * 1000)
        #elif(x in product_units for x in grams):
        #    price_per_unit = int(product_price.replace('₹','').replace(',',''))/int(product_quantity)
       # else:
        #    price_per_unit = int(product_price.replace('₹','').replace(',',''))
        row = [product_name, product_price, product_qty, prod_quantity, prod_units, price_per_unit, stor_name];
        d.append(row)
    
    dataset = pd.DataFrame(d, columns=["Product_Name", "Product_Price", "Product_Qty", "Prod_Quantity", "Prod_Units","Price_Per_Unit", "Store_Name"])

    return dataset

In [978]:
# get the data into a dataset
product_data = search_dailyneeds247()

In [979]:
product_data['Prod_Units'].unique()

array(['Grams', 'Kg', 'kg', 'Pouch', 'pieces', 'Nos', 'ml', 'Pack',
       'Litre', '1litre'], dtype=object)

In [980]:
product_data['Prod_Quantity'].unique()

array(['50', '200', '100', '500', '1', 'Grams', '400', '95', 'Kg', '7',
       '25', '10', 'Tin', '15', '5', '2', '80', '250', 'Oil', '150'],
      dtype=object)

In [981]:
print(product_data[product_data['Prod_Quantity'] == 'Grams'])
print(product_data[product_data['Prod_Quantity'] == 'Kg'])
print(product_data[product_data['Prod_Quantity'] == 'Oil'])
print(product_data[product_data['Prod_Quantity'] == 'Tin'])

                                                              Product_Name  \
43  Britannia Nutri Choice Cookies Thin Arrowroot Biscuits 150 Grams Pouch   

   Product_Price  Product_Qty Prod_Quantity Prod_Units Price_Per_Unit  \
43            22  Grams Pouch         Grams      Pouch                  

       Store_Name  
43  Dailyneeds247  
                           Product_Name Product_Price Product_Qty  \
95  Parrys White Label Sugar 1 Kg Pouch            60    Kg Pouch   

   Prod_Quantity Prod_Units Price_Per_Unit     Store_Name  
95            Kg      Pouch                 Dailyneeds247  
                   Product_Name Product_Price Product_Qty Prod_Quantity  \
158  Aachi Sunflower Oil 1litre           105  Oil 1litre           Oil   

    Prod_Units Price_Per_Unit     Store_Name  
158     1litre                 Dailyneeds247  
                   Product_Name Product_Price Product_Qty Prod_Quantity  \
124  Gold Ghee 1 Litre Tin Pack           570    Tin Pack           Tin   

 

In [982]:
product_data.drop(product_data.index[[43, 95,124, 158]], inplace=True)

In [983]:
grams = ["Gms", "Grams", "Ml", "Pc", "Pcs", "Pouch","Nos","ml"]
kilos = ["Kgs", "Kg", "kg", "Ltrs", "Litre","litre","1litre"]

for item in product_data.index:
    prod_qty = product_data['Prod_Quantity'][item]
    prod_unit = product_data['Prod_Units'][item]
    prod_price = product_data['Product_Price'][item]
    if any(x in prod_unit for x in kilos):
        product_data['Price_Per_Unit'][item] = int(product_price.replace('₹','').replace(',',''))/(int(prod_qty) * 1000)
    elif(x in prod_unit for x in grams):
        product_data['Price_Per_Unit'][item] = int(product_price.replace('₹','').replace(',',''))/(int(prod_qty))
    else:
        product_data['Price_Per_Unit'][item] = int(product_price.replace('₹','').replace(',',''))

In [993]:
product_data['Price_Per_Unit'] = product_data['Price_Per_Unit'].astype(float)
product_data['Product_Price'] = product_data['Product_Price'].astype(float)

In [994]:
product_data.to_csv('dailyneeds247.csv', index=False, encoding='utf-8')

In [995]:
dataset3 = product_data

In [996]:
dataset3.head()

,Product_Name,Product_Price,Product_Qty,Prod_Quantity,Prod_Units,Price_Per_Unit,Store_Name
0,Barley 50 Grams,7.0,50 Grams,50,Grams,0.8,Dailyneeds247
1,Kambu 200 Grams,9.0,200 Grams,200,Grams,0.2,Dailyneeds247
2,Barley 100 Grams,15.0,100 Grams,100,Grams,0.4,Dailyneeds247
3,Cumin Jeera Whole 50 Grams,20.0,50 Grams,50,Grams,0.8,Dailyneeds247
4,Sombu Sounf Green 50 Grams,20.0,50 Grams,50,Grams,0.8,Dailyneeds247


# Data Integration
Steps: 

Concatenate 3 datasets as we already matched the column names

Drop unwanted columns 

Fill null values


In [1011]:
# Combine datasets
Result_dataset = pd.concat([dataset1,dataset2,dataset3],ignore_index=True)

In [1012]:
# Drop unwanted columns
Result_dataset = Result_dataset.drop(['Links','Product_Qty','Prod_Units'], axis=1)
# Fill na values with 0 as other 2 platforms are not offering any discounts
Result_dataset[['Product_Discounted_Price','Discount_%']] = Result_dataset[['Product_Discounted_Price','Discount_%']].fillna(0)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332 entries, 0 to 1331
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Product_Name              1332 non-null   object 
 1   Prod_Quantity             1332 non-null   object 
 2   Product_Discounted_Price  1332 non-null   float64
 3   Product_Price             1332 non-null   float64
 4   Discount_%                1332 non-null   float64
 5   Price_Per_Unit            1332 non-null   float64
 6   Store_Name                1332 non-null   object 
dtypes: float64(4), object(3)
memory usage: 73.0+ KB


# Exploratory Data Analysis

Metadata information
Datatypes
Duplicates
Null values
Describe the data including object type

In [1049]:
# Summary of result dataset metadata
Result_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332 entries, 0 to 1331
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Product_Name              1332 non-null   object 
 1   Prod_Quantity             1332 non-null   object 
 2   Product_Discounted_Price  1332 non-null   float64
 3   Product_Price             1332 non-null   float64
 4   Discount_%                1332 non-null   float64
 5   Price_Per_Unit            1332 non-null   float64
 6   Store_Name                1332 non-null   object 
dtypes: float64(4), object(3)
memory usage: 73.0+ KB


In [1014]:
Result_dataset.describe()
# There are few products unit price is very high.

,Product_Discounted_Price,Product_Price,Discount_%,Price_Per_Unit
count,1332.000000,1332.000000,1332.000000,1332.000000
mean,74.554242,148.705143,1.130240,3.748133
std,280.574782,368.602824,4.920635,46.590674
min,0.000000,5.000000,0.000000,0.002667
25%,0.000000,35.000000,0.000000,0.125000
50%,0.000000,70.000000,0.000000,0.280000
75%,60.000000,139.000000,0.000000,0.680000
max,5600.000000,6900.000000,56.250000,1099.000000


In [1015]:
#looks safforn is in the top of the list seems costly item and ratnadeep is offering more products
Result_dataset.describe(include=object)

,Product_Name,Prod_Quantity,Store_Name
count,1332,1332.0,1332
unique,1279,118.0,3
top,Spar Mandi Loose Bestprice Pepper Black Whole,500.0,Ratnadeep
freq,3,174.0,660


In [1016]:
# Check organic products availability in differnet platforms
organic_products = Result_dataset[Result_dataset['Product_Name'].str.contains("organic",case=False)]

In [1017]:
organic_products
# Spar India  and Ratnadeep are offering organic products but spar offering more compare with other store. 

,Product_Name,Prod_Quantity,Product_Discounted_Price,Product_Price,Discount_%,Price_Per_Unit,Store_Name
32,"Organic Tattva - Organic Cashewsms,",100,210.0,210.0,0.00,2.100000,Spar India
64,Turn Organic - Organic Fenugreek Powder,100,35.0,35.0,0.00,0.350000,Spar India
65,Turn Organic - Organic Flaxseed,100,35.0,35.0,0.00,0.350000,Spar India
66,Organic Tattva - Garam Masala,100,160.0,160.0,0.00,1.600000,Spar India
72,Organic Tattva - Walnuts Karnels,100,275.0,275.0,0.00,2.750000,Spar India
75,24 Mantra - Organic Wheat Grass Powder,100,290.0,290.0,0.00,2.900000,Spar India
78,Turn Organic - Red Chilli Powder,100,50.0,50.0,0.00,0.500000,Spar India
81,Naturally Yours - Organic Ashwagandha Powder,100,140.0,140.0,0.00,1.400000,Spar India
87,Naturally Yours - Organic Neem Powder,100,105.0,105.0,0.00,1.050000,Spar India
94,Organic Tattva - Moong Dal Yellow Split,1000,250.0,250.0,0.00,0.250000,Spar India


In [1019]:
Discounts = Result_dataset[Result_dataset['Discount_%'] != 0]
Discounts.groupby(['Store_Name']).size()
# Spar India is offering discounts on 115 items but other 2 platforms not offering any.

Store_Name
Spar India    115
dtype: int64

In [1020]:
Result_dataset[Result_dataset['Price_Per_Unit'] >= 4]
# Safforn is teh only product which is very costlier and all other products price per gram is at normal level.
# Based on the price, it looks like outlier but based on the external sources it is not the outlier but kind of a costly or 
# rarely available product.

,Product_Name,Prod_Quantity,Product_Discounted_Price,Product_Price,Discount_%,Price_Per_Unit,Store_Name
0,Pina - Saffron,0.5,349.0,349.0,0.0,698.000000,Spar India
1,"Safe Harvest - Rice Flour,",1,69.0,69.0,0.0,69.000000,Spar India
3,Taj Mahal - Spanish Saffron,1,605.0,605.0,0.0,605.000000,Spar India
4,Kesari - Saffron Threads,1,375.0,375.0,0.0,375.000000,Spar India
5,Pina - Saffron,1,639.0,639.0,0.0,639.000000,Spar India
6,Pina - Saffron,1,1099.0,1099.0,0.0,1099.000000,Spar India
195,Taj-Mahal Spanish Saffron,2,1060.0,1060.0,0.0,530.000000,Spar India
458,Baby Corn,1,0.0,59.0,0.0,59.000000,Ratnadeep
471,Daily Fresh Poultry Eggs,30,0.0,206.0,0.0,6.866667,Ratnadeep
472,Ayur Plus Eggs,6,0.0,75.0,0.0,12.500000,Ratnadeep


In [1043]:
Top_Discounts = Result_dataset[Result_dataset['Discount_%'] >= 30]
Top_Discounts.sort_values(by='Discount_%',ascending =False)
# Based on the result set Keya brand products the category of masala the maximum discounts are available.
# More

,Product_Name,Prod_Quantity,Product_Discounted_Price,Product_Price,Discount_%,Price_Per_Unit,Store_Name
166,Elina Long Grain Rice,1000,70.0,160.0,56.25,0.07,Spar India
300,Spar - Brewed Natural Apple Cider Vinegar,370,199.0,390.0,48.97,0.54,Spar India
271,Disano - Extra Light Olive Oil,2000,1399.0,2395.0,41.59,0.70,Spar India
111,Disano - Extra Light Olive Oil,1000,749.0,1250.0,40.08,0.75,Spar India
11,"Keya - Chai Masalams,",100,133.0,190.0,30.00,1.33,Spar India
201,Keya - Kasuri Methi,20,42.0,60.0,30.00,2.10,Spar India
188,Keya - Awadhi Raita Masala,150,98.0,140.0,30.00,0.65,Spar India
186,Keya - Amritsari Chhole Masala,125,98.0,140.0,30.00,0.78,Spar India
185,Keya - Nawabi Meat Masala,125,112.0,160.0,30.00,0.90,Spar India
184,Keya - Chatpata Jaljeera Masala,125,98.0,140.0,30.00,0.78,Spar India


In [1032]:
def cost_est(product,qty):
    selected_products = Result_dataset[Result_dataset['Product_Name'].str.contains(product,case=False)]
    result_set = []

    for index,rows in selected_products.iterrows():
        product = rows['Product_Name']
        selected_quantity = qty
        Estimated_price = rows['Price_Per_Unit']*selected_quantity
#         discount = rows['Discount %']
        Store_Name = rows['Store_Name']
        result_set.append([product,selected_quantity,Estimated_price,Store_Name])

    data = pd.DataFrame(result_set,columns=['Selected Product','Selected Quantity','Price for selected quantity','Store_Name'])
    return data   

In [1046]:
if __name__=='__main__':
    select_product = input("Select any Product : ")
    select_qty = float(input("Enter the quantity in grams/ml: "))
    Recommended = cost_est(select_product,select_qty)
#Recommended.sort_values(by = 'Discount %',ascending=False)


Select any Product : oil
Enter the quantity in grams/ml: 1000


In [1025]:
selected_products = Result_dataset[Result_dataset['Product_Name'].str.contains('mix',case=False)]